<a href="https://colab.research.google.com/github/windaasm/Project-Newsletter/blob/main/Newsletter_Eclinic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from google.colab import auth
from google.auth import default
import gspread
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os
from io import BytesIO

# Authenticate with Google
auth.authenticate_user()
creds, _ = default()

def setup_google_services():
    """Setup Google Sheets and Drive services"""
    gc = gspread.authorize(creds)
    drive_service = build('drive', 'v3', credentials=creds)
    return gc, drive_service

def create_combo_chart(pendaftaran, utilisasi, selesai, row_num):
    """Generate combo chart dengan matplotlib"""
    labels = ["Juni", "Juli", "Agustus"]

    fig, ax1 = plt.subplots(figsize=(10, 6))
    fig.patch.set_facecolor('white')
    ax1.set_facecolor('white')

    x = np.arange(len(labels))
    width = 0.2

    bars1 = ax1.bar(x - width/2, pendaftaran, width, label='Pendaftaran', color='orange')
    bars2 = ax1.bar(x + width/2, selesai, width, label='Selesai Pelayanan', color='skyblue')

    ax1.set_xlabel('Bulan')
    ax1.set_ylabel('Jumlah Transaksi')
    ax1.set_xticks(x)
    ax1.set_xticklabels(labels)

    # Add arrows to axes
    ax1.annotate('', xy=(1, 0), xytext=(0, 0),
                xycoords='axes fraction', textcoords='axes fraction',
                arrowprops=dict(arrowstyle='->', lw=1.5, color='black'))
    ax1.annotate('', xy=(0, 1), xytext=(0, 0),
                xycoords='axes fraction', textcoords='axes fraction',
                arrowprops=dict(arrowstyle='->', lw=1.5, color='black'))

    # Hide top and right spines and set normal thickness
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['bottom'].set_linewidth(1.0)
    ax1.spines['left'].set_linewidth(1.0)

    ax2 = ax1.twinx()
    ax2.plot(x, utilisasi, color='red', marker='o', linewidth=2, markersize=8, label='Utilisasi (%)')
    ax2.set_ylabel('Utilisasi (%)')
    max_utilisasi = max(utilisasi) if utilisasi else 0
    ax2.set_ylim(0, max(100, max_utilisasi + 10))

    # Hide top spine for right axis and set normal thickness
    ax2.spines['top'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    ax2.spines['right'].set_linewidth(1.0)

    # Add arrow for right axis
    ax2.annotate('', xy=(1, 1), xytext=(1, 0),
                xycoords='axes fraction', textcoords='axes fraction',
                arrowprops=dict(arrowstyle='->', lw=1.5, color='black'))

    # Add border around entire figure
    from matplotlib.patches import Rectangle
    border = Rectangle((0.02, 0.02), 0.96, 0.96, linewidth=1, edgecolor='black',
                      facecolor='none', transform=fig.transFigure)
    fig.patches.append(border)

    # Get max bar height for positioning labels
    max_bar_height = max(max(pendaftaran), max(selesai))

    # Add data labels at bottom near x-axis
    y_min = ax1.get_ylim()[0]
    for i, bar in enumerate(bars1):
        ax1.text(bar.get_x() + bar.get_width()/2., y_min + (ax1.get_ylim()[1] - y_min) * 0.05,
                f'{int(pendaftaran[i])}', ha='center', va='bottom')

    for i, bar in enumerate(bars2):
        ax1.text(bar.get_x() + bar.get_width()/2., y_min + (ax1.get_ylim()[1] - y_min) * 0.05,
                f'{int(selesai[i])}', ha='center', va='bottom')

    for i, val in enumerate(utilisasi):
        ax2.text(x[i], val + 5, f'{val:.2f}%', ha='center', va='bottom')

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='lower center',
               bbox_to_anchor=(0.5, -0.25), ncol=3)

    plt.subplots_adjust(left=0.1, right=0.9, top=0.8, bottom=0.25)

    filename = f'combo_row_{row_num}.png'
    plt.savefig(filename, format='png', dpi=150, bbox_inches='tight',
                facecolor='white', edgecolor='none', pad_inches=0.3)
    plt.close()

    return filename

def upload_to_drive(drive_service, filename):
    """Upload file ke Google Drive"""
    folder_name = "Grafik Penggunaan Utilisasi"
    query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'"
    results = drive_service.files().list(q=query).execute()
    folders = results.get('files', [])

    if folders:
        folder_id = folders[0]['id']
    else:
        folder_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        folder = drive_service.files().create(body=folder_metadata).execute()
        folder_id = folder['id']

    file_metadata = {'name': filename, 'parents': [folder_id]}
    media = MediaFileUpload(filename, mimetype='image/png')
    file = drive_service.files().create(body=file_metadata, media_body=media).execute()

    drive_service.permissions().create(
        fileId=file['id'],
        body={'role': 'reader', 'type': 'anyone'}
    ).execute()

    return f"https://drive.google.com/uc?id={file['id']}"

# Main execution
gc, drive_service = setup_google_services()

# Replace with your actual spreadsheet ID from the Google Sheets URL
# Example: if URL is https://docs.google.com/spreadsheets/d/1ABC123xyz/edit
# then SPREADSHEET_ID = "1ABC123xyz"
SPREADSHEET_ID = "1VzZmI9hkI45c_BNMVEH6_fQPp8LVf6VFAUGLuUUsiSc"
SHEET_NAME = "Copy of Sheet1"

sh = gc.open_by_key(SPREADSHEET_ID)
worksheet = sh.worksheet(SHEET_NAME)
all_values = worksheet.get_all_values()

for row_idx in range(2, len(all_values)):
    row_num = row_idx + 1
    row_data = all_values[row_idx][17:26]

    if not any(row_data):
        continue

    pendaftaran = [float(x) if x else 0 for x in row_data[0:3]]
    utilisasi = [float(x) if x else 0 for x in row_data[3:6]]
    selesai = [float(x) if x else 0 for x in row_data[6:9]]

    filename = create_combo_chart(pendaftaran, utilisasi, selesai, row_num)
    direct_link = upload_to_drive(drive_service, filename)
    worksheet.update_cell(row_num, 27, direct_link)

    print(f"✅ Row {row_num}: {direct_link}")
    os.remove(filename)

print("🎉 Done!")

✅ Row 3: https://drive.google.com/uc?id=1jiVgT0CljboesrcXju1zVvPDh6hnQgpS
✅ Row 4: https://drive.google.com/uc?id=1MmNiBIqTh3Z-2hP-Q_2SLOs2zSxElgaV
✅ Row 5: https://drive.google.com/uc?id=1kK_TBZd7drUOCuk96pVSx0g2qcfvllbk
✅ Row 6: https://drive.google.com/uc?id=1K0RmsSv0w3epunknNLYrVxUMNZ0X1bb4
✅ Row 7: https://drive.google.com/uc?id=1O4ecCLfuwvvHUX1y8TEgmR-T_Gouvf5b
✅ Row 8: https://drive.google.com/uc?id=10cJhIxslpRphI7wp10WvA_SZW5eQ37j4
✅ Row 9: https://drive.google.com/uc?id=15JvgAT2oVsVpdC0qv5OUh5sa_AHQqtmp
✅ Row 10: https://drive.google.com/uc?id=16I5dKgCp0FdpDf3V_hld2zlnqM1xT65v
✅ Row 11: https://drive.google.com/uc?id=1JSVp4yfI23LMIzbbN3tLDB-nR8UzKniC
✅ Row 12: https://drive.google.com/uc?id=19VdlBDpLZ6zsHkAgFwrj_DnstZ6vH85L
✅ Row 13: https://drive.google.com/uc?id=14sd3gNozDwycXccowAqX11cP8jMUH2kG
✅ Row 14: https://drive.google.com/uc?id=1qnbVrlZ8WkfpXzlPB3CurpA211H7pPa6
✅ Row 15: https://drive.google.com/uc?id=1MRHE4HusK9mBSj-P2fH-l2iOvD6eKkkD
✅ Row 16: https://drive.google.c

In [20]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from google.colab import auth
from google.auth import default
import gspread
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os

# Authenticate with Google
auth.authenticate_user()
creds, _ = default()

def setup_google_services():
    """Setup Google Sheets and Drive services"""
    gc = gspread.authorize(creds)
    drive_service = build('drive', 'v3', credentials=creds)
    return gc, drive_service

def create_bar_chart(data, row_num):
    """Generate vertical bar chart dengan matplotlib"""
    labels = ["Juni 2025", "Juli 2025", "Agustus 2025"]

    fig, ax = plt.subplots(figsize=(4.5, 3))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    bars = ax.bar(labels, data, color='steelblue', width=0.6)

    ax.set_xlabel('Bulan', fontsize=8, fontweight='normal')
    ax.set_ylabel('Jumlah Fitur yang Sudah Digunakan', fontsize=8, fontweight='normal')
    ax.set_ylim(0, 9)
    ax.set_yticks(range(0, 10))

    # Set smaller font for axis labels
    ax.tick_params(axis='x', labelsize=8)
    ax.tick_params(axis='y', labelsize=8)

    # Add horizontal grid only
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5, axis='y')
    ax.set_axisbelow(True)

    # Add data labels on top of bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{int(data[i])}', ha='center', va='bottom',
                fontsize=8, fontweight='normal', color='black')

    # Remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.subplots_adjust(left=0.15, right=0.9, top=0.9, bottom=0.15)

    filename = f'bar_chart_row_{row_num}.png'
    plt.savefig(filename, format='png', dpi=150, bbox_inches=None,
                facecolor='white', edgecolor='none', pad_inches=0.3)
    plt.close()

    return filename

def upload_to_drive(drive_service, filename):
    """Upload file ke Google Drive"""
    folder_name = "Grafik Penggunaan Fitur"
    query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'"
    results = drive_service.files().list(q=query).execute()
    folders = results.get('files', [])

    if folders:
        folder_id = folders[0]['id']
    else:
        folder_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        folder = drive_service.files().create(body=folder_metadata).execute()
        folder_id = folder['id']

    file_metadata = {'name': filename, 'parents': [folder_id]}
    media = MediaFileUpload(filename, mimetype='image/png')
    file = drive_service.files().create(body=file_metadata, media_body=media).execute()

    drive_service.permissions().create(
        fileId=file['id'],
        body={'role': 'reader', 'type': 'anyone'}
    ).execute()

    return f"https://drive.google.com/uc?id={file['id']}"

# Main execution
gc, drive_service = setup_google_services()

# Replace with your actual spreadsheet ID
SPREADSHEET_ID = "1VzZmI9hkI45c_BNMVEH6_fQPp8LVf6VFAUGLuUUsiSc"
SHEET_NAME = "Copy of Sheet1"

sh = gc.open_by_key(SPREADSHEET_ID)
worksheet = sh.worksheet(SHEET_NAME)
all_values = worksheet.get_all_values()

for row_idx in range(2, len(all_values)):  # Start from row 3 (index 2)
    row_num = row_idx + 1
    row_data = all_values[row_idx][13:16]  # Columns N, O, P (index 13-15)

    # Skip empty rows
    if not any(row_data):
        continue

    # Convert to float, handle empty cells
    data = [float(x) if x else 0 for x in row_data]

    filename = create_bar_chart(data, row_num)
    direct_link = upload_to_drive(drive_service, filename)
    worksheet.update_cell(row_num, 17, direct_link)  # Column Q (index 17)

    print(f"✅ Row {row_num}: {direct_link}")
    os.remove(filename)

print("🎉 Semua row sudah diproses!")

✅ Row 3: https://drive.google.com/uc?id=1nMM8_NrHxmpM5X4rZEn-Do4lz0hrpyUe
✅ Row 4: https://drive.google.com/uc?id=155ZFBJXdiVxjPvi_usgwLuSqwqj53jUi
✅ Row 5: https://drive.google.com/uc?id=1_nWBUxt_U9o6N3FNoSApFf2g_WeLq7SY
✅ Row 6: https://drive.google.com/uc?id=16GlqTEnGfdPAj_z-gDnJWh3RxUUtsfq1
✅ Row 7: https://drive.google.com/uc?id=1UzVdgjSxrpF7WgnzIBOZtkEgFgn81igy
✅ Row 8: https://drive.google.com/uc?id=1vbDX8Xdq1ES8S1nClmbxRd1HP4khRya1
✅ Row 9: https://drive.google.com/uc?id=17a_p-i48U0z5Nuzl_aY_ypBalPQpH0Pr
✅ Row 10: https://drive.google.com/uc?id=1Q7vCHkUTaGQIM-lsJ7xvYaAkOyaq9qcV
✅ Row 11: https://drive.google.com/uc?id=14NdaTxm4nHmvThhcXFmjBoF8NPRRs6Pv
✅ Row 12: https://drive.google.com/uc?id=16liV6vNbro7lCI8eNyx4nDguAsTNT0kB
✅ Row 13: https://drive.google.com/uc?id=1SP6DI61WmVeshFOfrtYlLHIFqEOjByt0
✅ Row 14: https://drive.google.com/uc?id=1vXHN5o8soTlZBr9vB-X1zUYlOxXGjZDe
✅ Row 15: https://drive.google.com/uc?id=1HOtkkLRbVCE0Hp1g9EO4vGPH2XAeiQ27
✅ Row 16: https://drive.google.c